In [ ]:
import os
from os.path import join

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from bct.algorithms.modularity import community_louvain

In [ ]:
path_root = os.environ.get('DECIDENET_PATH')

path_derivatives = join(path_root, 'data/main_fmri_study/derivatives')
path_sourcedata = join(path_root, 'data/main_fmri_study/sourcedata') 

path_beh = join(path_sourcedata, 'behavioral')
path_bsc = join(path_derivatives, 'bsc')
path_nistats = join(path_derivatives, 'nistats')
path_parcellations = join(path_derivatives, 'parcellations')

path_corrmats = join(path_bsc, 'corrmats')

## Settings

In [ ]:
n_reps = 100
gamma = 1.5
B = 'negative_asym'
atlas = '300_ROI_Set'

In [ ]:
# Load data
corrmats_aggregated = np.load(join(path_corrmats, atlas, 
                                   'corrmats_aggregated.npy'))
n_subjects, n_conditions, n_perr_sign, n_rois, _ = corrmats_aggregated.shape

df_roi = pd.read_csv(join(path_corrmats, atlas, 'corrmats_aggregated.csv'))

In [ ]:
m_aggregated = np.zeros((n_subjects, n_conditions, n_perr_sign, n_rois))
q_aggregated = np.zeros((n_subjects, n_conditions, n_perr_sign))

for sub_idx in range(n_subjects):
    for con_idx in range(n_conditions):
        for perr_sign_idx in range(n_perr_sign):

            best_q = 0 
            for _ in range(n_reps):
                m, q = community_louvain(
                    corrmats_aggregated[sub_idx, con_idx, perr_sign_idx], 
                    B=B, 
                    gamma=gamma
                )
                if q > best_q:
                    best_m = m
                    best_q = q
            # Store best values
            m_aggregated[sub_idx, con_idx, perr_sign_idx] = best_m
            q_aggregated[sub_idx, con_idx, perr_sign_idx] = best_q

# Save results
np.save(join(path_corrmats, atlas, 'm_aggregated.npy'), 
        community_aggregated)
np.save(join(path_corrmats, atlas, 'q_aggregated.npy'), 
        q_aggregated)